# Excel to JSON Processor (Structured Data)

This notebook processes an Excel file and converts its content into a JSON file. It supports multiple sheets and allows the user to choose whether to process all sheets or a specific sheet.

### Step 1: Import Necessary Libraries

In [ ]:
%pip install pandas openpyxl

In [ ]:
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
import pandas as pd
import json
import os
import re
import datetime

### Step 2: Load the Excel File

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
import os

# --- Globals ---
selected_excel_path = None
excel_data = None

def select_file():
    global selected_excel_path, excel_data

    path = filedialog.askopenfilename(
        title="Select an Excel File",
        filetypes=(("Excel Files", "*.xlsx"), ("All Files", "*.*"))
    )

    if path:
        selected_excel_path = path
        label.config(text=os.path.basename(path))

        try:
            excel_data = pd.ExcelFile(path)
            preview = "\n".join(f"• {name}" for name in excel_data.sheet_names)

            text_widget.delete(1.0, tk.END)
            text_widget.insert(tk.END, f"Available Sheets:\n{preview}")
            button_confirm.pack(pady=10)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to load Excel:\n{e}")
            selected_excel_path = None
            label.config(text="No file selected")
            text_widget.delete(1.0, tk.END)
            button_confirm.pack_forget()
    else:
        label.config(text="No file selected")
        button_confirm.pack_forget()

def confirm_file():
    if selected_excel_path:
        window.destroy()
    else:
        messagebox.showwarning("No Selection", "Please select a file before confirming.")

# --- GUI Setup ---
window = tk.Tk()
window.title("Select an Excel File")
window.geometry("800x500+100+100")

label = tk.Label(window, text="No file selected", width=100)
label.pack(pady=20)

button_select = tk.Button(window, text="Select File", command=select_file)
button_select.pack(pady=10)

button_confirm = tk.Button(window, text="Load File", command=confirm_file)

text_widget = tk.Text(window, width=100, height=15)
text_widget.pack(pady=10)

window.mainloop()

# --- After GUI closes ---
if selected_excel_path:
    print(f"\n✅ Confirmed Excel file path: {selected_excel_path}")
    print(f"✅ Available Sheets: {excel_data.sheet_names}")
else:
    print("\n❌ No file selected.")


### Step 3: Prompt User for Sheet Selection

In [ ]:
if len(excel_data.sheet_names) > 1:
    process_all_sheets = input("Do you want to process all sheets? (Y/N): ").strip().lower()
    
    if process_all_sheets == "y":
        sheets_to_process = excel_data.sheet_names  # Process all sheets
    elif process_all_sheets == "n":
        sheet_name = input("Enter the sheet name to process: ").strip()
        if sheet_name not in excel_data.sheet_names:
            raise ValueError(f"Sheet '{sheet_name}' not found in the Excel file.")
        sheets_to_process = [sheet_name]  # Process only the specified sheet
    else:
        raise ValueError("Invalid input. Please enter 'Y' or 'N'.")
else:
    sheets_to_process = excel_data.sheet_names  # Process all sheets since there's only one sheet


### Step 4: Process the Selected Sheets

In [ ]:
# Define allowed characters (keep common symbols)
allowed_pattern = re.compile(r"[^\w\s.,!?;:'\"()\[\]\/-]")

# Unicode-safe strip function
def clean_string(text):
    # Remove unwanted symbols
    cleaned = allowed_pattern.sub('', text)
    # Strip leading/trailing spaces including non-breaking ones
    return cleaned.strip().replace('\u00A0', '').strip()

for sheet in sheets_to_process:
    print(f"Processing sheet: {sheet}")
    df = excel_data.parse(sheet)

    df = df.where(pd.notnull(df), None)

    json_data = df.to_dict(orient="records")

    # Clean each string in the record
    for record in json_data:
        for key, value in record.items():
            if isinstance(value, str):
                cleaned_value = clean_string(value.strip())
                if ';' in cleaned_value:
                    record[key] = [item.strip() for item in cleaned_value.split(';') if item.strip()]
                else:
                    record[key] = cleaned_value

    print(f"✅ JSON data for '{sheet}' written successfully:")
    print(pd.DataFrame(json_data))

### Step 5: Save the JSON Data to a File

In [ ]:
excel_filename_only = os.path.splitext(os.path.basename(selected_excel_path))[0]  # Remove folder + extension
output_excel_file_path = f"../../data/processed/processed_{excel_filename_only}.json"

# --- Ensure output directory exists ---
os.makedirs(os.path.dirname(output_excel_file_path), exist_ok=True)

# --- Custom serializer for datetime ---
def custom_serializer(obj):
    if isinstance(obj, datetime.datetime):
        return obj.isoformat()  # ISO 8601 format
    raise TypeError(f"Type {type(obj)} not serializable")

# --- Save JSON ---
with open(output_excel_file_path, "w", encoding="utf-8") as json_file:
    json.dump(json_data, json_file, indent=4, ensure_ascii=False, default=custom_serializer)

print(f"✅ JSON data saved to {output_excel_file_path}")

### - END